In [330]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
from tensorflow import keras
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import matplotlib.pyplot as plt

Mix of https://towardsdatascience.com/multi-layer-perceptron-using-tensorflow-9f3e218a4809 and https://www.easy-tensorflow.com/tf-tutorials/neural-networks/two-layer-neural-network
Also, mainly https://nasirml.wordpress.com/2017/12/16/multi-layer-perceptron-in-tensorflow-part-2-mnist/

Loading dataset with a function. No need i guess really

In [331]:
def load_dataset():
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

    # normalize x
    X_train = tf.keras.utils.normalize(X_train, axis = 1)
    X_test = tf.keras.utils.normalize(X_test, axis = 1)

    # we reserve the last 10000 training examples for validation
    X_train, X_val = X_train[:-10000], X_train[-10000:]
    y_train, y_val = y_train[:-10000], y_train[-10000:]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [332]:
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()
## Printing dimensions
print(X_train.shape, len(y_train),X_val.shape, len(y_val), X_test.shape, y_test.shape)
## Visualizing the first digit
#plt.imshow(X_train[0], cmap="Greys");

(50000, 28, 28) 50000 (10000, 28, 28) 10000 (10000, 28, 28) (10000,)


We need to make arrays of 784, not 28x28 and One-Hot encoding for labels

In [333]:
## Changing dimension of input images from N*28*28 to  N*784
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1]*X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1]*X_test.shape[2]))

print('Train dimension:');print(X_train.shape)
print('Test dimension:');print(X_test.shape)

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('Train labels dimension:');print(y_train.shape)
print('Test labels dimension:');print(y_test.shape)

Train dimension:
(50000, 784)
Test dimension:
(10000, 784)
Train labels dimension:
(50000, 10)
Test labels dimension:
(10000, 10)


In [334]:
# Network parameters
num_features = X_train.shape[1]    # 784, units in the input layer 28x28 images
num_hidden1 = 512              # units in the first hidden layer
num_hidden2 = 256
#prob num_output not necessary since already defined num_classes=10 above#
num_output = y_train.shape[1]   # 10, units in the output layer 0 to 9. OR nClasses
starter_learning_rate = 0.001

In [335]:
# tf Graph input i.e. Placeholders for the input data and labels 
input_X = tf.placeholder('float32',shape =(None,num_features),name="input_X")
input_y = tf.placeholder('float32',shape = (None,num_classes),name='input_Y')

In [336]:
#Instead, use this
## Weights initialized by random normal function with std_dev = 1/sqrt(number of input features)
weights_0 = tf.Variable(tf.random_normal([num_features,num_hidden1], stddev=(1/tf.sqrt(float(num_features)))))
bias_0 = tf.Variable(tf.random_normal([num_hidden1]))

weights_1 = tf.Variable(tf.random_normal([num_hidden1,num_hidden2], stddev=(1/tf.sqrt(float(num_hidden1)))))
bias_1 = tf.Variable(tf.random_normal([num_hidden2]))

weights_2 = tf.Variable(tf.random_normal([num_hidden2,num_output], stddev=(1/tf.sqrt(float(num_hidden2)))))
bias_2 = tf.Variable(tf.random_normal([num_output]))

In [337]:
## Initializing weigths and biases
hidden_output_0 = tf.nn.relu(tf.matmul(input_X,weights_0)+bias_0)

hidden_output_1 = tf.nn.relu(tf.matmul(hidden_output_0,weights_1)+bias_1)

predicted_y = tf.sigmoid(tf.matmul(hidden_output_1,weights_2) + bias_2)

In [338]:
# cost function and optimization
loss_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predicted_y, labels=input_y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss_func)

In [339]:
## Variable learning rate
learning_rate = tf.train.exponential_decay(starter_learning_rate, 0, 5, 0.85, staircase=True)

In [340]:
## Metrics definition
correct_prediction = tf.equal(tf.argmax(y_train,1), tf.argmax(predicted_y,1), name='correct_pred')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = 'accuracy')

In [341]:
batch_size = 200
epochs=14

training_accuracy = []
training_loss = []
testing_accuracy = []

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):    
    arr = np.arange(X_train.shape[0])
    np.random.shuffle(arr)
    for index in range(0,X_train.shape[0],batch_size):
        sess.run(optimizer, {input_X: X_train[arr[index:index+batch_size]],
                          input_y: y_train[arr[index:index+batch_size]]})
        training_accuracy.append(sess.run(accuracy, feed_dict= {input_X:X_train, 
                                                         input_y: y_train}))
        training_loss.append(sess.run(loss_func, {input_X: X_train, 
                                      input_y: y_train}))
    
     ## Evaluation of model
    testing_accuracy.append(accuracy_score(y_test.argmax(1), 
                            sess.run(model, {input_X: X_test}).argmax(1)))
    print("Epoch: {0}, Train loss: {1:.2f} Train acc: {2:.3f}, Test acc:{3:.3f}".format(epoch, training_loss[epoch], training_accuracy[epoch], testing_accuracy[epoch]))

InvalidArgumentError: You must feed a value for placeholder tensor 'input_X_17' with dtype float and shape [?,784]
	 [[node input_X_17 (defined at <ipython-input-308-b67b4c241fae>:2) ]]

Original stack trace for 'input_X_17':
  File "C:\Users\adria\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\adria\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\adria\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\adria\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\adria\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "C:\Users\adria\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\adria\Anaconda3\lib\asyncio\base_events.